# VacationPy
---

## Import Libraries and Load the Weather and Coordinates Data

In [56]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [57]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/oliverkisza/Desktop/Bootcampfiles/python-api-challenge-6/WeatherPy/output_data/cities.csv")
#i have no idea why this would not work without using absolute path
# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,qaqortoq,60.7167,-46.0333,280.98,75,86,2.00,GL,1691700549
1,1,petropavlovsk-kamchatsky,53.0452,158.6483,282.48,87,0,4.00,RU,1691700549
2,2,punta arenas,-53.1500,-70.9167,276.55,72,100,6.81,CL,1691700289
3,3,itoman,26.1247,127.6694,300.58,94,20,3.60,JP,1691700549
4,4,invercargill,-46.4000,168.3500,274.83,91,100,2.68,NZ,1691700469



### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [58]:
# Configure the map plot
city_map = city_data_df.hvplot.points(x='Lng', y='Lat', size='Humidity', tiles='OSM', alpha=0.5, markersize=city_data_df['Humidity'], hover_cols=['City', 'Humidity']).opts(width=800, height=500, title='City Humidity')

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [59]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities = city_data_df[(city_data_df['Max Temp'] > 290) &(city_data_df['Max Temp'] < 305) & (city_data_df['Cloudiness'] == 3)]

# Drop any rows with null values
filtered_cities = filtered_cities.dropna()

# Display sample data
filtered_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
88,88,tiksi,71.6872,128.8694,290.12,49,3,2.14,RU,1691700393
164,164,bataipora,-22.2953,-53.2711,303.64,26,3,2.15,BR,1691700577
363,363,huarmey,-10.0681,-78.1522,294.66,74,3,5.87,PE,1691700617
482,482,enguera,38.9797,-0.6868,300.42,86,3,1.60,ES,1691700640


### Step 3: Create a new DataFrame called `hotel_df`.

In [60]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
88,tiksi,RU,71.6872,128.8694,49,
164,bataipora,BR,-22.2953,-53.2711,26,
363,huarmey,PE,-10.0681,-78.1522,74,
482,enguera,ES,38.9797,-0.6868,86,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [63]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"distance(lat,lng,{lat},{lng})<={radius}"
    params["bias"] = f"distance,latlng,{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tiksi - nearest hotel: No hotel found
bataipora - nearest hotel: No hotel found
huarmey - nearest hotel: No hotel found
enguera - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
88,tiksi,RU,71.6872,128.8694,49,No hotel found
164,bataipora,BR,-22.2953,-53.2711,26,No hotel found
363,huarmey,PE,-10.0681,-78.1522,74,No hotel found
482,enguera,ES,38.9797,-0.6868,86,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [76]:
# Configure the map plot
map_plot = hotel_df.hvplot.points(x='Lng', y='Lat', size='Humidity', tiles='OSM', alpha=0.5, markersize='Humidity', color='green', hover_cols=['City', 'Country', 'Hotel Name', 'Humidity']).opts(width=800, height=500, tools=['hover'], title='Filtered City Humidity')

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)